## NoSQL 데이터베이스 MongoDB

### 몽고디비 - 데이터를 도큐먼트 타입으로 저장하는 NoSQL

- BJON(Binary JSON)의 형태로 저장
- 스키마를 지정하지 않아도 됨(유연성)
- 클러스터를 지원해 스케일아웃이 쉬움
- 메모리를 많이 사용
- 트랜잭션 지원이 RDB에 비해 약함

### 몽고디비 CRUD API 생성

In [ ]:
// 몽고디비 예제
const MongoClient=require('mongodb').MongoClient;
const url=require('./Info.js');

// MongoClient 생성
const client=new MongoClient(url, {useNewUrlParser: true});

async function main(){
    try{
        // 커넥션을 생성하고 연결 시도
        await client.connect();
        console.log('MongoDB 접속 성공');

        // test 데이터베이스의 person 컬렉션 가져오기
        const collection=client.db('test').collection('person');

        // 문서 하나 추가
        await collection.insertOne({name: 'Andy', age: 30});
        console.log('문서 추가 완료')

        // 문서 찾기
        const documents=await collection.find({name: 'Andy', age: 30}).toArray();
        console.log('찾은 문서: ', documents);

        // 문서 갱신하기
        await collection.updateOne({name: 'Andy'}, {$set: {age: 31}} );
        console.log('문서 업데이트 완료')

        // 갱신된 문서 확인하기
        const updatedDocuments=await collection.find({name: 'Andy'}).toArray();
        console.log('갱신된 문서: ', updatedDocuments);

        // 문서 삭제하기
        // await collection.deleteOne({name: 'Andy'});
        // console.log('문서 삭제')

        // 연결 끊기
        await client.close();
    }catch (err) {
        console.error(err);
    }
}

main();

Promise { <pending> }

(node:23104) [MONGODB DRIVER] Warning: useNewUrlParser is a deprecated option: useNewUrlParser has no effect since Node.js Driver version 4.0.0 and will be removed in the next major version
(Use `node --trace-warnings ...` to show where the warning was created)


MongoDB 접속 성공
문서 추가 완료
찾은 문서:  [
  {
    _id: new ObjectId('6812e0b1be95e0443f142fd5'),
    name: 'Andy',
    age: 30
  }
]
문서 업데이트 완료
갱신된 문서:  [
  {
    _id: new ObjectId('6812e0b1be95e0443f142fd5'),
    name: 'Andy',
    age: 31
  }
]


### 몽구스를 사용해 CRUD API 생성

In [1]:
// 몽구스는 객체를 도큐먼트로 매핑하는 기능 보유, 스키마 지정가능
// 기존 몽고디비 예제의 person 객체를 스키마로 생성
var mongoose=require('mongoose');
var Schema=mongoose.Schema;

const personSchema=new Schema({     // 스키마 객체 생성
    name: String,
    age: Number,
    email: {type: String, required: true},
});

// module.exports=mongoose.model('Person', personSchema);  // 모델 객체 생성

In [1]:
// CRUD를 테스트할 익스프레스 API 코드
const express=require("express");
const bodyParser=require("body-parser");
const mongoose=require("mongoose");
const Schema=mongoose.Schema;
const personSchema=new Schema({ 
    name: String,
    age: Number,
    email: {type: String, required: true},
});
const Person = mongoose.model("Person", personSchema);

mongoose.set("strictQuery", false);     // 경고가 뜨지 않게 설정

const app=express();
app.use(bodyParser.json());     // HTTP에서 Body를 파싱하기 위한 설정
app.listen(3000, async() => {
    console.log("server started");
    const mongodbUri=require('./Info.js');      
    
    // 몽고디비에 커넥션 맺기
    mongoose
        .connect(mongodbUri, {useNewUrlParser: true})
        .then(console.log("Connected to MongoDB"));
});

// 모든 person 데이터 출력
app.get("/person", async(req, res) => {
    const person=await Person.find({});
    res.send(person);
});

// 특정 이메일로 person 찾기
app.get("/person/:email", async(req, res) => {
    const person=await Person.findOne({email: req.params.email});
    res.send(person);
});

// person 데이터 추가하기
app.post("/person", async (req, res) => {
    const person=new Person(req.body);      
    await person.save();        // const result=await Person.create(req.body);
                                // res.send(result); 와 같은 코드
    res.send(person);
});

// person 데이터 수정하기
app.put("/person/:email", async (req, res) => {
    const person=await Person.findOneAndUpdate(
        {email: req.params.email},
        {$set: req.body},
        {new: true}
    );
    console.log(person);
    res.send(person);
});

// person 데이터 삭제하기
app.delete("/person/:email", async (req, res) => {
    await Person.deleteMany({email: req.params.email});
    res.send({success: true});
})


<ref *1> [Function: app] {
  _events: [Object: null prototype] { mount: [Function: onmount] },
  _eventsCount: 1,
  _maxListeners: undefined,
  setMaxListeners: [Function: setMaxListeners],
  getMaxListeners: [Function: getMaxListeners],
  emit: [Function: emit],
  addListener: [Function: addListener],
  on: [Function: addListener],
  prependListener: [Function: prependListener],
  once: [Function: once],
  prependOnceListener: [Function: prependOnceListener],
  removeListener: [Function: removeListener],
  off: [Function: removeListener],
  removeAllListeners: [Function: removeAllListeners],
  listeners: [Function: listeners],
  rawListeners: [Function: rawListeners],
  listenerCount: [Function: listenerCount],
  eventNames: [Function: eventNames],
  init: [Function: init],
  defaultConfiguration: [Function: defaultConfiguration],
  handle: [Function: handle],
  use: [Function: use],
  route: [Function: route],
  engine: [Function: engine],
  param: [Function: param],
  set: [Function

server started
Connected to MongoDB


(node:30736) [MONGODB DRIVER] Warning: useNewUrlParser is a deprecated option: useNewUrlParser has no effect since Node.js Driver version 4.0.0 and will be removed in the next major version
(Use `node --trace-warnings ...` to show where the warning was created)
ValidationError: Person validation failed: email: Path `email` is required.
    at Document.invalidate (d:\study\NODEJS\node_modules\mongoose\lib\document.js:3343:32)
    at d:\study\NODEJS\node_modules\mongoose\lib\document.js:3104:17
    at d:\study\NODEJS\node_modules\mongoose\lib\schemaType.js:1407:9
    at process.processTicksAndRejections (node:internal/process/task_queues:77:11)


{
  _id: new ObjectId('68130ab7f0cdd3f570621652'),
  name: 'Andy Park',
  age: 32,
  email: 'andy@backend.com',
  __v: 0
}


### REST 클라이언트로 API 테스트
- rest client extension 설치